# Datensätze miteinander verbinden

In [1]:
import pandas as pd

Schauen wir uns einen Ausschnitt aus einer Umfrage bei allen Schulleiter*innen in NRW an, die gerade im WDR durchgeführt wird. 

In [2]:
df = pd.read_csv('./data/umfrage_schulleiter.csv')

In [3]:
df

,Ort,schulform,anzahl_schueler,F1_ausstattung
0,Aachen,Privatschule,178,3.0
1,Aachen,Gymnasium,NaN,4.0
2,Aachen,Gymnasium,1250,3.0
3,Aachen,Realschule,640,5.0
4,Aachen,Grundschule,195,4.0
...,...,...,...,...
768,Wuppertal,Förderschule,203,4.0
769,Würselen,Grundschule,209,5.0
770,Würselen,Weiterbildungskolleg,310,4.0
771,Xanten,Berufskolleg,365,2.0


## Wo vergeben Schulleiter*innen in NRW die besten Noten für die technische Ausstattung an ihrer Schule?
Unter f1_ausstattung haben die Schulleiter*innen Schulnoten für die Ausstattung ihrer Schule vergeben.
Im Datensatz gibt es nur die Spalte 'Ort', die einen Gemeindenamen enthält. 
Um die Daten nach Region analysieren und auch später visualisieren zu können, wäre es hilfreich, weitere Angaben wie den AGS, Regierungsbezirk etc. zu haben.

### Allgemeine NRW-Daten einladen

In [4]:
# Laden der generellen Daten zu den NRW-Gemeinden 
city_data = pd.read_json('./data/nrw_data.json', orient='columns', dtype={'keyCity': 'str'})

In [5]:
city_data.sample()

,city,type,keyCity,state,studioRadio,studio,region,district
213,Lüdenscheid,Stadt,5962032,Nordrhein-Westfalen,Siegen,Siegen,Westfalen,Märkischer Kreis


KeyCity enthält in diesem Fall den AGS ohne führende Null.

Wir wollen aus dem AGS die Regierungsbezirke-Namen ablesen, dafür erzeigen wir eine neue Spalte 'regierungsbezirk', die die erste Stelle aus dem AGS enthält.

In [6]:
city_data['regierungsbezirk'] = city_data.keyCity.str[1:2]

In [7]:
city_data.sample()

,city,type,keyCity,state,studioRadio,studio,region,district,regierungsbezirk
387,Windeck,Gemeinde,5382076,Nordrhein-Westfalen,Köln,Bonn,Rheinland,Rhein-Sieg-Kreis,3


In [8]:
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('1', 'Düsseldorf')
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('3', 'Köln')
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('5', 'Münster')
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('7', 'Detmold')
city_data['regierungsbezirk'] = city_data.regierungsbezirk.str.replace('9', 'Arnsberg')

In [9]:
city_data.regierungsbezirk.value_counts()

Köln          99
Arnsberg      83
Münster       78
Detmold       70
Düsseldorf    66
Name: regierungsbezirk, dtype: int64

### Datensätze verbinden

In [10]:
# Um die Datensätze zu verbinden, setzen wir in beiden Fällen den Index auf die Ort-Spalte
df = df.set_index('Ort')

In [11]:
df.sample()

,schulform,anzahl_schueler,F1_ausstattung
Ort,,,
Lemgo,Berufskolleg,1200,2.0


In [12]:
city_data = city_data.set_index('city')

In [13]:
city_data.sample()

,type,keyCity,state,studioRadio,studio,region,district,regierungsbezirk
city,,,,,,,,
Moers,Stadt,5170024,Nordrhein-Westfalen,Duisburg,Duisburg,Rheinland,Wesel,Düsseldorf


In [14]:
# Um zu kontrollieren, dass keine Spalten verloren gehen, überprüfen wir die Länge der beiden Datensätze
len(df), len(city_data)

(773, 396)

In [15]:
# Join verbindet beide Datensätze anhand des index
df = df.join(city_data)

In [16]:
len(df)

773

In [17]:
df.sample(10)

,schulform,anzahl_schueler,F1_ausstattung,type,keyCity,state,studioRadio,studio,region,district,regierungsbezirk
Düren,Grundschule,300,5.0,Stadt,5358008,Nordrhein-Westfalen,Aachen,Aachen,Rheinland,Düren,Köln
Aachen,Grundschule,130,4.0,Stadt,5334002,Nordrhein-Westfalen,Aachen,Aachen,Rheinland,Städteregion Aachen,Köln
Velbert,Förderschule,136,2.0,Stadt,5158032,Nordrhein-Westfalen,Wuppertal,Wuppertal,Rheinland,Mettmann,Düsseldorf
Ahaus,Gesamtschule,1214,NaN,Stadt,5554004,Nordrhein-Westfalen,Münster,Münster,Westfalen,Borken,Münster
Elsdorf,Förderschule,140,5.0,Stadt,5362016,Nordrhein-Westfalen,Köln,Köln,Rheinland,Rhein-Erft-Kreis,Köln
Wermelskirchen,Grundschule,175,2.0,Stadt,5378032,Nordrhein-Westfalen,Wuppertal,Wuppertal,Rheinland,Rheinisch-Bergischer Kreis,Köln
Bonn,Grundschule,278,4.0,Stadt,5314000,Nordrhein-Westfalen,Köln,Bonn,Rheinland,Bonn,Köln
Viersen,Gesamtschule,1265,4.0,Stadt,5166032,Nordrhein-Westfalen,Düsseldorf,Düsseldorf,Rheinland,Viersen,Düsseldorf
Neuenrade,Sekundarschule,400,5.0,Stadt,5962048,Nordrhein-Westfalen,Siegen,Siegen,Westfalen,Märkischer Kreis,Arnsberg
Neuss,Grundschule,302,4.0,Stadt,5162024,Nordrhein-Westfalen,Düsseldorf,Düsseldorf,Rheinland,Rhein-Kreis Neuss,Düsseldorf


#### Tipp: 

Es gibt noch weitere Arten, Datensätze miteinander zu verknüpfen, einen ersten Überblick findest du hier:
        [Chris Albon: Join And Merge Pandas Dataframe](https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/)

### Wie bewerten Sie die digitale Ausstattung Ihrer Schule zurzeit?

In [18]:
# Gibt es Schulen, die diese Frage nicht beantwortet haben
df.F1_ausstattung.value_counts(dropna=False)

3.0    185
5.0    177
2.0    159
4.0    152
6.0     58
1.0     32
NaN     10
Name: F1_ausstattung, dtype: int64

In [19]:
df[df.F1_ausstattung.isna()]

,schulform,anzahl_schueler,F1_ausstattung,type,keyCity,state,studioRadio,studio,region,district,regierungsbezirk
Ahaus,Gesamtschule,1214,NaN,Stadt,5554004,Nordrhein-Westfalen,Münster,Münster,Westfalen,Borken,Münster
Bielefeld,Grundschule,NaN,NaN,Stadt,5711000,Nordrhein-Westfalen,Bielefeld,Bielefeld,Westfalen,Bielefeld,Detmold
Bocholt,Berufskolleg,378,NaN,Stadt,5554008,Nordrhein-Westfalen,Münster,Münster,Westfalen,Borken,Münster
Düsseldorf,NaN,NaN,NaN,Stadt,5111000,Nordrhein-Westfalen,Düsseldorf,Düsseldorf,Rheinland,Düsseldorf,Düsseldorf
Gelsenkirchen,NaN,NaN,NaN,Stadt,5513000,Nordrhein-Westfalen,Essen,Essen,Ruhrgebiet,Gelsenkirchen,Münster
Herne,Grundschule,NaN,NaN,Stadt,5916000,Nordrhein-Westfalen,Essen,Essen,Ruhrgebiet,Herne,Arnsberg
Köln,Förderschule,164,NaN,Stadt,5315000,Nordrhein-Westfalen,Köln,Köln,Rheinland,Köln,Köln
Mettmann,Grundschule,260,NaN,Stadt,5158024,Nordrhein-Westfalen,Wuppertal,Düsseldorf,Rheinland,Mettmann,Düsseldorf
Windeck,Gymnasium,650,NaN,Gemeinde,5382076,Nordrhein-Westfalen,Köln,Bonn,Rheinland,Rhein-Sieg-Kreis,Köln
Windeck,Grundschule,103,NaN,Gemeinde,5382076,Nordrhein-Westfalen,Köln,Bonn,Rheinland,Rhein-Sieg-Kreis,Köln


## Durchschnitt aller Schulen

In [20]:
# Pandas hat eine eigene Funktion, um Durchschnoittswerte zu bilden
df.F1_ausstattung.mean()

3.5989515072083877

### Zufriedenheit nach Regierungsbezirken 

In [21]:
# Um zu ermitteln, in welchem Regierungsbezirk die Schulleiter*innen am zufriedensten sind, nutzen wir group_by: 
df.groupby('regierungsbezirk').mean()

,F1_ausstattung
regierungsbezirk,
Arnsberg,3.765823
Detmold,3.381443
Düsseldorf,3.676647
Köln,3.601093
Münster,3.407407


### Übung

Schreibe die jeweilige group_by - Funktion

In [22]:
# Zufriedenheit nach Stadt / Land (type)
df.groupby('type').mean()

,F1_ausstattung
type,
Gemeinde,3.196721
Stadt,3.633436


In [23]:
# Zufriedenheit nach Regionen
df.groupby('region').mean()

,F1_ausstattung
region,
Rheinland,3.609231
Ruhrgebiet,3.896970
Westfalen,3.354260


In [24]:
# Zufriedenheit nach Landkreisen
df.groupby('district').mean().sort_values(by='F1_ausstattung', ascending=False)

,F1_ausstattung
district,
Krefeld,5.000000
Hagen,4.857143
Bochum,4.681818
Leverkusen,4.500000
Essen,4.500000
Hamm,4.333333
Mönchengladbach,4.153846
Bonn,4.142857
Märkischer Kreis,4.000000


### Übung

In [25]:
# Nach Schulform
df.groupby('schulform').mean().sort_values(by='F1_ausstattung', ascending=False)

,F1_ausstattung
schulform,
PRIMUS-Schule,5.000000
Hauptschule,4.130435
Grundschule,4.009174
Förderschule,3.672414
Sonstiges,3.600000
Realschule,3.393443
Sekundarschule,3.350000
Gesamtschule,3.278481
Weiterbildungskolleg,3.142857


Wo gibt es aus deiner Sicht die spannendsten Ergebnisse?

### Übung

Wie findest Du heraus, wie viele Schulen eine 1 bzw. eine 6 vergeben haben?

In [26]:
# Wieviele Schulen sind super zufrieden (1.0)
len(df[df.F1_ausstattung == 1.0])

32

In [27]:
# Wieviele Schulen sind super unzufrieden (6.0)
len(df[df.F1_ausstattung == 6.0])

58

Auf zur nächsten Datenquelle: 
    
**[ >> Excel-Dokumente Notebook](06b_excel.ipynb)**